In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

In [2]:
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options

In [3]:
url = 'https://sportsbook.draftkings.com/leagues/basketball/nba'

In [4]:
options = Options()
options.add_argument('--headless')
driver = Firefox(options=options)

driver.get(url)

html = driver.page_source
soup = bs(html, 'lxml')

driver.close()

In [5]:
table = soup.find('tbody', {'class': 'sportsbook-table__body'})
rows = table.findAll('tr')
games, game, idx = [], {'visitor': '', 'local': '', 'v_line': 0, 'l_line': 0}, 0
for row in rows:
    team_name = row.find('div', {'class': 'event-cell__name-text'}).text.split(' ')
    if len(team_name[0]) == 3:
        team_id = team_name[0]
    else:
        team_id = team_name[0] + team_name[1][0]
    odd = row.find('span', {'class': 'sportsbook-odds american no-margin default-color'}).text.replace('−', '-')
    
    if idx == 0:
        game['visitor'] = team_id
        game['v_line'] = int(odd)
        idx = 1
        
    elif idx == 1:
        game['local'] = team_id
        game['l_line'] = int(odd)
        idx = 0
        
        games.append(game)
        game = {'visitor': '', 'local': '', 'v_line': 0, 'l_line': 0}
    
games_df = pd.DataFrame(games)
games_df

,visitor,local,v_line,l_line
0,LAL,BOS,500,-700
1,IND,NYK,124,-148
2,CLE,MEM,-325,260
3,PHI,UTA,160,-192


In [6]:
games_df.to_excel('Models/games_odds.xlsx', index=0)